In [20]:
import os
import sys

print("Current working directory:", os.getcwd())
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    print(f"Added {parent_dir} to Python path")
else:
    print(f"{parent_dir} already in Python path")

Current working directory: /Users/jinceyang/Desktop/codebase/ml/learn_d2l/chap7
/Users/jinceyang/Desktop/codebase/ml/learn_d2l already in Python path


In [21]:
import importlib
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import numpy as np

from d2l.base.function import corr2d, corr2d_multi_in, corr2d_multi_in_out, corr2d_multi_in_out_1x1, comp_conv2d, max_pool2d, avg_pool2d

In [22]:
X = torch.tensor([
    [0.0, 1.0, 2.0], 
    [3.0, 4.0, 5.0], 
    [6.0, 7.0, 8.0]
])

K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [23]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [24]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [25]:
K = torch.tensor([[1.0, -1.0]])

In [26]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [27]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [28]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2).
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2 # Learning rate

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 1.262
epoch 4, loss 0.212
epoch 6, loss 0.036
epoch 8, loss 0.006
epoch 10, loss 0.001


In [29]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9943, -0.9940]])

In [30]:
conv2d = nn.LazyConv2d(1, kernel_size=3)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([6, 6])

In [31]:
# 1 row and column is padded on either side, so a total of 2 rows or columns
# are added
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [32]:
conv2d = nn.LazyConv2d(1, kernel_size=(5, 3))
comp_conv2d(conv2d, X).shape

torch.Size([4, 6])

In [33]:
# We use a convolution kernel with height 5 and width 3. The padding on either
# side of the height and width are 2 and 1, respectively
conv2d = nn.LazyConv2d(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [34]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1)
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [35]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [36]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

In [37]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

AssertionError: 

In [ ]:
X = torch.tensor([
    [0.0, 1.0, 2.0], 
    [3.0, 4.0, 5.0], 
    [6.0, 7.0, 8.0]
])
max_pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [39]:
X

tensor([[[ 1.1733,  1.3541,  0.0945],
         [ 0.2059, -1.2956, -2.5080],
         [-1.6739,  1.0939, -0.3329]],

        [[-1.8227, -1.5565, -1.1200],
         [-0.1059,  0.0611,  2.1049],
         [ 0.1984, -1.1872, -0.3115]],

        [[ 1.2726,  0.0655,  1.3341],
         [-2.3716,  2.8314,  0.3498],
         [ 1.1726,  1.1761,  1.1181]]])

In [38]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[1.3541, 1.3541],
         [1.0939, 1.0939]],

        [[0.0611, 2.1049],
         [0.1984, 2.1049]],

        [[2.8314, 2.8314],
         [2.8314, 2.8314]]])

In [40]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[1.3541, 1.3541],
         [1.0939, 1.0939]],

        [[0.0611, 2.1049],
         [0.1984, 2.1049]],

        [[2.8314, 2.8314],
         [2.8314, 2.8314]]])

In [41]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[ 1.1733,  1.3541,  0.0945],
         [ 0.2059, -1.2956, -2.5080],
         [-1.6739,  1.0939, -0.3329],
         [ 2.1733,  2.3541,  1.0945],
         [ 1.2059, -0.2956, -1.5080],
         [-0.6739,  2.0939,  0.6671]],

        [[-1.8227, -1.5565, -1.1200],
         [-0.1059,  0.0611,  2.1049],
         [ 0.1984, -1.1872, -0.3115],
         [-0.8227, -0.5565, -0.1200],
         [ 0.8941,  1.0611,  3.1049],
         [ 1.1984, -0.1872,  0.6885]],

        [[ 1.2726,  0.0655,  1.3341],
         [-2.3716,  2.8314,  0.3498],
         [ 1.1726,  1.1761,  1.1181],
         [ 2.2726,  1.0655,  2.3341],
         [-1.3716,  3.8314,  1.3498],
         [ 2.1726,  2.1761,  2.1181]]])

In [42]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[1.3541, 1.3541],
         [2.3541, 2.3541],
         [2.3541, 2.3541]],

        [[0.0611, 2.1049],
         [0.1984, 2.1049],
         [1.1984, 3.1049]],

        [[2.8314, 2.8314],
         [2.8314, 2.8314],
         [3.8314, 3.8314]]])